# ETL I

## Task I - Define correct schema for json data

* load json dataset
* look at the infered schema
* define the schema explicitly
* see what happens if the schema is defined wrong

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, explode, split, regexp_replace, collect_list

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('ETL I')
    .getOrCreate()
)

In [ ]:
spark.version

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

data_input_path = os.path.join(project_path, 'data/questions-json')

output_path = os.path.join(project_path, 'output/questions-transformed')

<b>First let Spark infer the schema:</b>

In [ ]:
questionsDF = (
    spark
    .read
    .format('json')
    .option('path', data_input_path)
    .load()
)

Note:
Consider loading only one file to check the schema. Loading the json with lots of files without specifying schema takes a lot of time.

In [ ]:
questionsDF.printSchema()

<b>Now define the schema:</b>

In [ ]:
json_schema = StructType(
    [
        StructField('question_id', LongType(), True),
        StructField('creation_date', TimestampType(), True),
        StructField('title', StringType(), True),
        StructField('body', StringType(), True),
        StructField('tags', StringType(), True),
        StructField('accepted_answer_id', LongType(), True),
        StructField('answers', LongType(), True),
        StructField('comments', LongType(), True),
        StructField('user_id', LongType(), True),
        StructField('views', LongType(), True),
    ]
)

In [ ]:
questionsDF = (
    spark
    .read
    .schema(json_schema)
    .format('json')
    .option('path', data_input_path)    
    .load()
)

In [ ]:
questionsDF.show(truncate=10)

#### What happens if:

* question_id is TimestampType in the defined schema
* title is the TimestampType in the defined schema

Hint
* Try the `mode` argument to he load function:
    * where mode is one of the following:
    * FAILFAST
    * DROPMALFORMED
    * PERMISSIVE (default)


In [ ]:
questionsDF = (
    spark
    .read
    .schema(json_schema)
    .format('json')
    .option('path', data_input_path)    
    .load(mode='None')
)

### Try the following
* set the defined schema wrong (title as LongType)
* keep the mode as default (None or Permissive)
* run these two queries:
    * questionsDF.filter(col('question_id').isNull()).count()
    * questionsDF.filter(col('question_id').isNull()).show()
    
Can you explain what is happening?

In [ ]:
questionsDF.filter(col('question_id').isNull()).count()

In [ ]:
questionsDF.filter(col('question_id').isNull()).show()

## Task II - Transfrom json to parquet and convert String column to an array

* convert column tags to array of tags 

#### Convert tags to an array

Hint
* use split to get an array
* explode the array
* use regexp_replace
* groupBy + collect_list
* join with original questions DataFrame

In [ ]:
resultDF = (
    questionsDF
    .withColumn('tags_arr', split('tags', '><'))
    .withColumn('tag', explode('tags_arr'))
    .withColumn('tag', regexp_replace('tag', '(<|>)', ''))
    .groupBy('question_id')
    .agg(collect_list('tag').alias('tags'))
    .join(questionsDF.drop('tags'), 'question_id')
)

#### Save the data

Hint:

* repartition the data to 8 partitions before saving
 * this will create 8 files

In [ ]:
(
    resultDF
    .repartition(8)
    .write
    .mode('overwrite')
    .option('path', output_path)
    .save()
)

<b>Check if we saved the data correctly:</b>

In [ ]:
checkDF = (
    spark
    .read
    .parquet(output_path)
)

In [ ]:
checkDF.count()

In [ ]:
checkDF.show(truncate=5)

In [ ]:
checkDF.select('tags').show(truncate=False)

In [ ]:
spark.stop()